In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from underthesea import word_tokenize


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import re

import transformers
from transformers import AutoTokenizer

from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from rank_bm25 import BM25Okapi

from utils.retrieval import BM25Retrieval, TFIDFRetrieval, SentenceRetrieval
device

'cuda'

In [2]:
bm25 = BM25Retrieval()
tfidf = TFIDFRetrieval()
sentences = SentenceRetrieval()

In [5]:
def split_context(context):
    docs = re.split(r'(?<!\d)\.\s*(?![.])', context)
    docs = [doc for doc in docs if doc]
    return docs

In [6]:
def get_top_rate(query, context, rate=0.2, option=bm25, isProcessbar=False):
    if isProcessbar:
        global pbar
        pbar.update(1)
    docs =split_context(context)
    top_rate = option.get_top_rate(query, docs, rate=rate)
    k = min(5, len(top_rate))
    return '\n'.join(top_rate[:k])

def get_topk(query, context, k=5, option=tfidf, isProcessbar=False):
    if isProcessbar:
        global pbar
        pbar.update(1)
    docs =split_context(context)
    return '\n'.join(option.get_topk(query, docs, k=k))

In [7]:
df = pd.read_csv('data/train_v2.csv')

In [8]:
pbar = tqdm(total=len(df)*2)
df['top_tfdif'] = df.apply(lambda x: get_topk(x['claim_tokenizer'], x['context_tokenizer'], option=tfidf, k=5, isProcessbar=True), axis=1)
# df['top_similarity'] = df.apply(lambda x: get_top_rate(x['claim_tokenizer'], x['context_tokenizer'], option=sentences, rate=0.2, isProcessbar=True), axis=1)
df['top_bm25'] = df.apply(lambda x: get_topk(x['claim_tokenizer'], x['context_tokenizer'], option=bm25, k=5, isProcessbar=True), axis=1)

  0%|          | 0/75142 [00:00<?, ?it/s]

In [9]:
# df = df[df['top_similarity']!='']
df = df.reset_index(drop=True)

In [9]:
# df['top_similarity'] = df['top_similarity'].apply(lambda x: x.lower())
df['claim_tokenizer'] = df['claim_tokenizer'].apply(lambda x: x.lower())
df['context_tokenizer'] = df['context_tokenizer'].apply(lambda x: x.lower())
df['top_tfdif'] = df['top_tfdif'].apply(lambda x: x.lower())

In [10]:
df

,context,claim,verdict,evidence,context_tokenizer,claim_tokenizer,top_tfdif,top_bm25
0,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","ngoài việc không giới_hạn_mức lương , công_ty ...","công_ty cũng có chế_độ đãi_ngộ tốt , có lương ...","công_ty cũng có chế_độ đãi_ngộ tốt , có lương ..."
1,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","thành_lập năm 2016 , phát_triển phần_mềm , tổ_...","bên cạnh đó , faba thường_xuyên tổ_chức những ...","bên cạnh đó , faba thường_xuyên tổ_chức những ..."
2,"phát biểu tại lễ ký kết vào ngày 17/11, giám đ...","nhằm phát triển đội ngũ, faba thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","nhằm phát_triển đội_ngũ , faba thường tổ_chức ...","bên cạnh đó , faba thường_xuyên tổ_chức những ...","bên cạnh đó , faba thường_xuyên tổ_chức những ..."
3,"đối với các nhà khoa học, trí thức, điều kiện ...",sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...",sở_dĩ vị_trí kỹ_sư phần_mềm có mức lương cao n...,""" không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ...",""" không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ..."
4,"đối với các nhà khoa học, trí thức, điều kiện ...","theo adeco việt nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...","theo adeco việt_nam , mức lương cao nhất đối_v...","theo adeco việt_nam , mức lương cao nhất đối_v...","theo adeco việt_nam , mức lương cao nhất đối_v..."
...,...,...,...,...,...,...,...,...
37566,"""tôi cảm nhận mình đang sống, đang tự do"", bia...",thái lan không dự kiến đón ít nhất 5 triệu khá...,REFUTED,Thái Lan dự kiến đón ít nhất 5 triệu khách Tru...,""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",thái_lan không dự_kiến đón ít_nhất 5 triệu khá...,thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...,thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...
37567,"""tôi cảm nhận mình đang sống, đang tự do"", bia...","trước đại dịch, người trung quốc chiếm gần 1/2...",REFUTED,"Trước đại dịch, người Trung Quốc chiếm gần 1/3...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...","trước đại_dịch , người trung_quốc chiếm gần 1/...","trước đại_dịch , người trung_quốc chiếm gần 1/...","trước đại_dịch , người trung_quốc chiếm gần 1/..."
37568,"""tôi cảm nhận mình đang sống, đang tự do"", bia...",zhu nói rằng số tiền 290.000 usd không thể mua...,REFUTED,"""Số tiền đó chỉ có thể mua được căn hộ nhỏ ở v...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",zhu nói rằng số tiền 290.000 usd không_thể mua...,""" số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ...",""" số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ..."
37569,"""tôi không có đầu óc đầu tư chứng khoán và đan...","tiền tiết kiệm của các hộ gia đình giảm 9,9 ng...",REFUTED,"Tiền tiết kiệm của các hộ gia đình tăng 9,9 ng...",""" tôi không có đầu_óc đầu_tư chứng_khoán và đa...","tiền tiết_kiệm của các hộ gia_đình giảm 9,9_ng...","tiền tiết_kiệm của các hộ gia_đình tăng 9,9_ng...","tiền tiết_kiệm của các hộ gia_đình tăng 9,9_ng..."


In [11]:
label2id = {'SUPPORTED': 0, 'REFUTED': 1, 'NEI': 2}
id2label = {0: 'SUPPORTED', 1: 'REFUTED', 2: 'NEI'}
df['verdict_label'] = df['verdict'].map(label2id)

In [12]:
df.top_tfdif.values

array(['công_ty cũng có chế_độ đãi_ngộ tốt , có lương tháng 13 , 14 , phụ_cấp ..\nđể thu_hút và giữ_chân nhân_tài trong một thị_trường nhân_sự it khốc_liệt , faba không giới_hạn_mức lương , luôn cập_nhật để dải lương đạt mức cạnh_tranh tốt so với thị_trường \nông cũng chia_sẻ việc trở_thành đối_tác đào_tạo và tuyển_dụng cùng funix sẽ giúp faba technology đạt mục_tiêu trở_thành công_ty offshore_development chất_lượng nhất trong khu_vực \ntrong đó có havas - một trong những tập_đoàn quảng_cáo và truyền_thông lớn nhất thế_giới \nqua hợp_tác này , faba kỳ_vọng sẽ có ứng_viên chất_lượng cho rất nhiều vị_trí công_việc đang tìm_kiếm như automation tester , ui / ux designer , web developer , mobile developer ở nhiều cấp_độ ..',
       'bên cạnh đó , faba thường_xuyên tổ_chức những dự_án đào_tạo kỹ_năng nghề_nghiệp , kỹ_năng mềm , chú_trọng xây_dựng văn_hóa doanh_nghiệp để thúc_đẩy phát_triển đội_ngũ \nfaba technology thành_lập năm 2016 , chuyên cung_cấp dịch_vụ phát_triển phần_mềm offshore cho

In [13]:
df.to_csv('data/train.csv', index=False)

In [ ]:
df['top_similarity']

0        công_ty cũng có chế_độ đãi_ngộ tốt , có lương ...
1        faba technology thành_lập năm 2016 , chuyên cu...
2        bên_cạnh đó , faba thường_xuyên tổ_chức những ...
3        " không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ...
4        theo adeco việt_nam , mức lương cao nhất đối_v...
                               ...                        
37283    thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...
37284    trước đại_dịch , người trung_quốc chiếm gần 1/...
37285    " số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ...
37286    tiền tiết_kiệm của các hộ gia_đình tăng 9,9_ng...
37287    cô cho_hay đã tham_gia vào nhiều cuộc đấu_giá ...
Name: top_similarity, Length: 37288, dtype: object

In [14]:
df

,context,claim,verdict,evidence,context_tokenizer,claim_tokenizer,top_similarity,verdict_label,top_tfdif,top_bm25
0,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","ngoài việc không giới_hạn_mức lương , công_ty ...","công_ty cũng có chế_độ đãi_ngộ tốt , có lương ...",0,"công_ty cũng có chế_độ đãi_ngộ tốt , có lương ...","công_ty cũng có chế_độ đãi_ngộ tốt , có lương ..."
1,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","thành_lập năm 2016 , phát_triển phần_mềm , tổ_...","faba technology thành_lập năm 2016 , chuyên cu...",0,"bên_cạnh đó , faba thường_xuyên tổ_chức những ...","bên_cạnh đó , faba thường_xuyên tổ_chức những ..."
2,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...","phát_biểu tại lễ ký_kết vào ngày 17/11 , giám_...","nhằm phát_triển đội_ngũ , faba_thường tổ_chức ...","bên_cạnh đó , faba thường_xuyên tổ_chức những ...",0,"bên_cạnh đó , faba thường_xuyên tổ_chức những ...","bên_cạnh đó , faba thường_xuyên tổ_chức những ..."
3,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...",sở_dĩ vị_trí kỹ_sư phần_mềm có mức lương cao n...,""" không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ...",0,""" không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ...",""" không riêng kỹ_sư phần_mềm , đối_với bất_kỳ ..."
4,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...","đối_với các nhà_khoa_học , trí_thức , điều_kiệ...","theo adeco việt_nam , mức lương cao nhất đối_v...","theo adeco việt_nam , mức lương cao nhất đối_v...",0,"theo adeco việt_nam , mức lương cao nhất đối_v...","theo adeco việt_nam , mức lương cao nhất đối_v..."
...,...,...,...,...,...,...,...,...,...,...
37283,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Thái Lan không dự kiến đón ít nhất 5 triệu khá...,REFUTED,Thái Lan dự kiến đón ít nhất 5 triệu khách Tru...,""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",thái_lan không dự_kiến đón ít_nhất 5 triệu khá...,thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...,1,thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...,thái_lan dự_kiến đón ít_nhất 5 triệu khách tru...
37284,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...","Trước đại dịch, người Trung Quốc chiếm gần 1/2...",REFUTED,"Trước đại dịch, người Trung Quốc chiếm gần 1/3...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...","trước đại_dịch , người trung_quốc chiếm gần 1/...","trước đại_dịch , người trung_quốc chiếm gần 1/...",1,"trước đại_dịch , người trung_quốc chiếm gần 1/...","trước đại_dịch , người trung_quốc chiếm gần 1/..."
37285,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Zhu nói rằng Số tiền 290.000 USD không thể mua...,REFUTED,"""Số tiền đó chỉ có thể mua được căn hộ nhỏ ở v...",""" tôi cảm_nhận mình đang sống , đang tự_do "" ,...",zhu nói rằng số tiền 290.000 usd không_thể mua...,""" số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ...",1,""" số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ...",""" số tiền đó chỉ có_thể mua được căn_hộ nhỏ ở ..."
37286,"""Tôi không có đầu óc đầu tư chứng khoán và đan...","Tiền tiết kiệm của các hộ gia đình giảm 9,9 ng...",REFUTED,"Tiền tiết kiệm của các hộ gia đình tăng 9,9 ng...",""" tôi không có đầu_óc đầu_tư chứng_khoán và đa...","tiền tiết_kiệm của các hộ gia_đình giảm 9,9_ng...","tiền tiết_kiệm của các hộ gia_đình tăng 9,9_ng...",1,"tiền tiết_kiệm của các hộ gia_đình tăng 9,9_ng...","tiền tiết_kiệ